In [ ]:
import astropy.coordinates as coord
import astropy.table as at
import astropy.units as u
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

# gala
import gala.coordinates as gc
import gala.dynamics as gd
import gala.integrate as gi
import gala.potential as gp
from gala.units import galactic

from pyia import GaiaData
from dustmaps.bayestar import BayestarQuery

The original 2:

http://simbad.harvard.edu/simbad/sim-id?bibyear1=1850&bibyear2=%24currentYear&submit=Display&Ident=%40602764&Name=BD%2B40++2790&bibdisplay=refsum&bibyear1=1850&bibyear2=%24currentYear#lab_bib

http://simbad.harvard.edu/simbad/sim-id?Ident=%402119260&Name=BD%2b39%20%202643&submit=submit

In [ ]:
g1 = GaiaData.from_query("SELECT * FROM gaiadr2.gaia_source WHERE source_id=1490845584382687232;")
g2 = GaiaData.from_query("SELECT * FROM gaiadr2.gaia_source WHERE source_id=1476485996883837184;")

**ARCES exposure time**

In [ ]:
# from arcesetc import signal_to_noise_to_exp_time

# # sptype = 'K3V'
# sptype = 'K2V'
# wavelength = 6563 * u.Angstrom
# signal_to_noise = 30
# V = 9.97

# signal_to_noise_to_exp_time(sptype, wavelength, V, signal_to_noise).to(u.min)

---

In [ ]:
c1 = g1.get_skycoord()[0]
c2 = g2.get_skycoord()[0]

In [ ]:
c1.ra.degree, c1.dec.degree

In [ ]:
c2.ra.degree, c2.dec.degree

In [ ]:
g1.parallax, g2.parallax

In [ ]:
c1.get_constellation()

In [ ]:
c2.get_constellation()

In [ ]:
c1

In [ ]:
c1.separation(c2)

In [ ]:
c1.pm_ra_cosdec, c1.pm_dec

In [ ]:
c2.pm_ra_cosdec, c2.pm_dec

In [ ]:
np.sqrt((c1.pm_ra_cosdec - c2.pm_ra_cosdec)**2 + (c1.pm_dec - c2.pm_dec)**2)

In [ ]:
(c1.velocity - c2.velocity).norm()

In [ ]:
g1.parallax_error, g2.parallax_error

In [ ]:
s1 = g1.get_error_samples(size=100_000).get_skycoord()
s2 = g2.get_error_samples(size=100_000).get_skycoord()
dv = (s1.velocity - s2.velocity).norm()
dx = (s1.data.to_cartesian() - s2.data.to_cartesian()).norm()

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(10 ,5))

axes[0].hist(dx.to_value(u.pc)[0], np.linspace(8.5, 10, 128));
axes[0].set_xlabel(rf'$\Delta x$ [{u.pc:latex_inline}]')

axes[1].hist(dv.to_value(u.km/u.s)[0], np.linspace(0, 3, 128));
axes[1].set_xlabel(rf'$\Delta v$ [{u.km/u.s:latex_inline}]')

fig.set_facecolor('w')
fig.tight_layout()

In [ ]:
mw = gp.MilkyWayPotential()
_ = coord.galactocentric_frame_defaults.set('v4.0')

In [ ]:
w1 = gd.PhaseSpacePosition(
    g1.get_error_samples(size=1_000).get_skycoord().transform_to(coord.Galactocentric()).data)
w2 = gd.PhaseSpacePosition(
    g2.get_error_samples(size=1_000).get_skycoord().transform_to(coord.Galactocentric()).data)

In [ ]:
orbit1 = mw.integrate_orbit(w1, dt=-0.2*u.Myr, t1=0, t2=-100*u.Myr,
                            Integrator=gi.DOPRI853Integrator)
orbit2 = mw.integrate_orbit(w2, t=orbit1.t,
                            Integrator=gi.DOPRI853Integrator)

In [ ]:
# _ = orbit1[:, :10].plot()
# _ = orbit2[:, :10].plot()

In [ ]:
dx = (orbit1.pos - orbit2.pos).norm()
dx.shape

In [ ]:
fig = plt.figure()
plt.plot(orbit1.t.to_value(u.Myr), dx.to_value(u.pc),
         marker='', alpha=0.1, color='k')
plt.ylim(0, 50)
plt.xlim(-100, 0)
plt.xlabel('time [Myr]')
plt.ylabel(r'$\Delta x$ [pc]')
fig.set_facecolor('w')

Check for alignment of ∆x and ∆v:

In [ ]:
dxnorm = orbit1.pos / orbit1.pos.norm()
dvnorm = orbit1.vel / orbit1.vel.norm()

In [ ]:
plt.plot(orbit1.t, dxnorm[:, 0].dot(dvnorm[:, 0]))

---

Other stars on the sky:

In [ ]:
g = GaiaData.from_query("""
SELECT * FROM gaiadr2.gaia_source as gaia
WHERE parallax > 15
AND radial_velocity IS NOT NULL
AND 1=CONTAINS(
  POINT('ICRS', gaia.ra, gaia.dec),
  CIRCLE('ICRS', 210, 39.79039914276253, 120)
)
""")

In [ ]:
len(g)

In [ ]:
gplx = g[(g.parallax_over_error > 6) & np.isfinite(g.radial_velocity)]
len(gplx)

In [ ]:
cc = gplx.get_skycoord()

In [ ]:
dx = cc.data.to_cartesian() - c1.data.to_cartesian()
dv = cc.velocity - c1.velocity

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(5, 5))

ax.scatter(dx.norm().to_value(u.pc), 
           dv.norm().to_value(u.km/u.s),
           lw=0, s=8, alpha=0.5)

ax.set(xscale='log',
       yscale='log',
       xlim=(1e-1, 100),
       ylim=(1e-1, 10),
       xlabel=r'$\Delta x$ [pc]',
       ylabel=rf'$\Delta v$ [{u.km/u.s:latex_inline}]');

ax.axhline(3.)

In [ ]:
(0.65*u.km/u.s * 40*u.Myr).to(u.pc)

In [ ]:
derp = gplx[(dv.norm() < 3*u.km/u.s) & 
            (dx.norm() < 80*u.pc) & 
            (gplx.distance > 30*u.pc) & (gplx.distance < 50*u.pc)]
derp.data['source_id', 'ra', 'dec', 'parallax', 'phot_g_mean_mag'].pprint(max_width=100000)

In [ ]:
ebv = derp.get_ebv()

In [ ]:
# bprp = derp.phot_bp_mean_mag - derp.phot_rp_mean_mag
# gmag = derp.phot_g_mean_mag

bprp = derp.get_BP0() - derp.get_RP0()
gmag = derp.get_G0()

In [ ]:
cs = plt.scatter(derp.ra.value, derp.dec.value,
                 c=derp.radial_velocity_error,
                 vmin=0, vmax=1, zorder=100, lw=0.5, 
                 edgecolor='#555555', 
                 s=2 * (15 - derp.phot_g_mean_mag.value))
#                  c=1000/derp.parallax.value,
#                  vmin=40, vmax=60, lw=0.5, 
#                  edgecolor='#555555')
plt.xlim(100, 300)
plt.ylim(0, 80)
plt.colorbar(cs)

In [ ]:
# [Fe/H] = -0.07 from Bochanski
iso = at.Table.read('/Users/apricewhelan/data/Isochrones/MIST/FeH_0.0_iso.fits')
iso = iso[iso['phase'] < 1]
iso = iso[(iso['log10_isochrone_age_yr'] > 7.5) & (iso['log10_isochrone_age_yr'] < 8.3)]

In [ ]:
fig, ax = plt.subplots(figsize=(6, 6))
# ax.scatter(bprp, gmag,
#            c=1000/derp.parallax.value,
#            vmin=40, vmax=60, zorder=100, lw=0.5, 
#            edgecolor='#555555')

cc = ax.scatter(bprp, gmag,
                c=derp.radial_velocity_error,
                vmin=0, vmax=1, zorder=100, lw=0.5, 
                edgecolor='#555555')
fig.colorbar(cc)

for age in np.unique(iso['log10_isochrone_age_yr']):
    i = iso[iso['log10_isochrone_age_yr'] == age]
    ax.plot(i['G_BP'] - i['G_RP'], 
            i['G'] + g1.distmod.value - 0.1,
            marker='', alpha=0.5, label=f'{10**age / 1e6:.0f} Myr')

ax.set(xlabel='BP-RP',
       ylabel='G',
       xlim=(0., 2.5),
       ylim=(13, 5))

ax.legend()
fig.set_facecolor('w')

This one is a no!

In [ ]:
hd140667 = derp[gmag < 8*u.mag]
hd140667_c = hd140667.get_skycoord()

hd140667_c = coord.SkyCoord(ra=hd140667_c.ra, dec=hd140667_c.dec,
                            distance=hd140667_c.distance,
                            pm_ra_cosdec=hd140667_c.pm_ra_cosdec,
                            pm_dec=hd140667_c.pm_dec,
                            radial_velocity=-16.727*u.km/u.s) # SB9 COM velocity

In [ ]:
(hd140667_c.velocity - derp_c.velocity).norm()

In [ ]:
hd140667_w = gd.PhaseSpacePosition(hd140667_c.transform_to(coord.Galactocentric()).data)
hd_orbit = mw.integrate_orbit(hd140667_w, t=orbit1.t)

Rewind all orbits?

In [ ]:
derp_w = gd.PhaseSpacePosition(
    derp.get_error_samples(size=1_000).get_skycoord().transform_to(coord.Galactocentric()).data)

In [ ]:
derp_orbit = mw.integrate_orbit(derp_w, t=orbit1.t)

In [ ]:
derp_dx = (orbit1.pos[:, None] - derp_orbit.pos.reshape((len(orbit1.t),) + derp_w.shape)).norm()
# derp_dx = (hd_orbit.pos[:, None, None] - derp_orbit.pos.reshape((len(orbit1.t),) + derp_w.shape)).norm()
derp_dx.shape

In [ ]:
_ = orbit1[:, 0].plot()

In [ ]:
for i in range(derp_dx.shape[1]):
    fig = plt.figure()
    plt.plot(orbit1.t.to_value(u.Myr), 
             derp_dx[:, i].to_value(u.pc),
             marker='', alpha=0.1, color='k')
    plt.ylim(0, 50)
    plt.xlim(-100, 0)
    plt.xlabel('time [Myr]')
    plt.ylabel(r'$\Delta x$ [pc]')
    fig.set_facecolor('w')

## Any APOGEE data?

In [ ]:
rave = at.Table.read('/Users/apricewhelan/data/RAVE/rave_gaia_xmatch.fits')

In [ ]:
lamost = at.Table.read('/Users/apricewhelan/data/LAMOST/DDPayne_LAMOST-DR5_recommend.fits')

In [ ]:
allstar = at.Table.read('/Users/apricewhelan/data/APOGEE_beta/apogee_astroNN-r13-l33-58932beta.fits')

In [ ]:
lamost_c = coord.SkyCoord(lamost['RA'], lamost['DEC'], unit='deg')

In [ ]:
derp_c = derp.get_skycoord()
lamost_idx, lamost_sep, _ = derp_c.match_to_catalog_sky(lamost_c)
lamost_data = lamost[lamost_idx[lamost_sep < 2*u.arcsec]]
lamost_data['source_id'] = derp.data[lamost_sep < 2*u.arcsec]['source_id']

In [ ]:
derp.data['has_apogee'] = np.isin(derp.source_id, allstar['source_id'])
derp.data['has_rave'] = np.isin(derp.source_id, rave['source_id'])
derp.data['has_lamost'] = (lamost_sep < 2*u.arcsec)

In [ ]:
subtbl = derp.data['source_id', 'ra', 'dec', 'phot_g_mean_mag', 'has_apogee', 'has_lamost', 'has_rave']
subtbl[np.argsort(subtbl['phot_g_mean_mag'])].pprint(max_width=100000)

In [ ]:
stars = allstar[np.isin(allstar['source_id'], derp.source_id)]
stars['source_id', 'FE_H', 'FE_H_ERR', 'MG_H', 'SI_H', 'age']

In [ ]:
fig, ax = plt.subplots(figsize=(6, 6))
ax.scatter(bprp, gmag,
           c=derp.radial_velocity_error,
           vmin=0, vmax=1, zorder=100, lw=0.5, 
           edgecolor='#555555')
#            c=1000/derp.parallax.value,
#            vmin=40, vmax=50, zorder=100, lw=0.5, 
#            edgecolor='#555555')

ax.scatter(bprp[derp.has_apogee], 
           gmag[derp.has_apogee], zorder=200, lw=2, 
           facecolor='none', color='tab:purple', s=60)

ax.scatter(bprp[derp.has_lamost], 
           gmag[derp.has_lamost], zorder=200, lw=2, 
           facecolor='none', color='tab:green', s=100)

for age in np.unique(iso['log10_isochrone_age_yr']):
    i = iso[iso['log10_isochrone_age_yr'] == age]
    ax.plot(i['G_BP'] - i['G_RP'], 
            i['G'] + g1.distmod.value - 0.1,
            marker='', alpha=0.5, label=f'{10**age / 1e6:.0f} Myr')

ax.set(xlabel='BP-RP',
       ylabel='G',
       xlim=(0.5, 3),
       ylim=(14, 5))

ax.legend()
fig.set_facecolor('w')

In [ ]:
lamost_data

In [ ]:
(10**i[i['G'] < 12.5]['log_Teff']).min()

---

In [ ]:
from astropy.time import Time

In [ ]:
loc = coord.EarthLocation.of_site('APO')

In [ ]:
t = Time.now() + np.linspace(0, 16, 128)*u.hour
altaz = derp_c[None].transform_to(coord.AltAz(obstime=t,
                                              location=loc)[:, None])

In [ ]:
plt.plot((t - 8*u.hr).datetime, altaz.secz)
plt.ylim(0.5, 2)
plt.xticks(rotation=45)